In [1]:
%%time
from selenium import webdriver
from PIL import Image
from io import BytesIO
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import requests

shop_url = 'https://www.wconcept.co.kr/Product/300504814?gclid=Cj0KCQjwrdjnBRDXARIsAEcE5YkPD5JkB8VQ6qzvET-3mwpJsMNb41q7doMAsx4usAyIj6VkADQomMgaApWLEALw_wcB'
shop_url2 = 'http://www.halfclub.com/Detail?PrstCd=RT9STS05&ColorCd=ZZ9&NaPm=ct%3Djwiyxnns%7Cci%3Db1398f8848980fa78f7c352048e8c70e72a08ff7%7Ctr%3Dligh%7Csn%3D270%7Chk%3Dfdee3d0bae83774a4617e196f46087ad4a8c458a'
shop_url3 = 'http://www.tmon.co.kr/deal/1959358378?NaPm=ct%3Djwk3obtc%7Cci%3D265e98e52a8d47dd798cb5e73c5c831654b8f5f6%7Ctr%3Dslsl%7Csn%3D221844%7Chk%3D4c2172339f11fa4d15e6b65e6108253d62b67cdf&utm_source=naver&utm_medium=affiliate&utm_term=&utm_content=&utm_campaign=%EB%84%A4%EC%9D%B4%EB%B2%84%EC%A7%80%EC%8B%9D%EC%87%BC%ED%95%91_%ED%86%B5%ED%95%A9%EA%B2%80%EC%83%89%ED%95%AB%EB%94%9C'
verbose = 1
driver_path = './geckodriver.exe'
browser = webdriver.Firefox(executable_path=driver_path)
browser.get(shop_url3)

# from here http://stackoverflow.com/questions/1145850/how-to-get-height-of-entire-document-with-javascript
js = 'return Math.max( document.body.scrollHeight, document.body.offsetHeight,  document.documentElement.clientHeight,  document.documentElement.scrollHeight,  document.documentElement.offsetHeight);'

scrollheight = browser.execute_script(js)
print(f'scrollheight : {scrollheight}')
scale = 0.8
browser.execute_script(f'document.body.style.MozTransform = "scale({scale})";')

scrollheight : 208876
Wall time: 8.95 s


In [3]:
html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')
# 모든 img 태그 가지고옴 
imgs = soup.find_all('img')
img_src = []
for img_ in imgs:
    img_src.append(img_.get('src'))

In [25]:
len(img_src)

483

In [24]:
import requests
import shutil
# user-agent를 설정해줘야 로봇이 아닌줄 알고 막지 않음
# user-agent 설정 안해주면 403 error 발생
r = requests.get(img_src[0], stream=True, headers={'User-agent': 'Mozilla/5.0'})
if r.status_code == 200:
    with open("./img/img.jpg", 'wb') as f:
        r.raw.decode_content = True
        shutil.copyfileobj(r.raw, f)
else:
    print(r.status_code)

In [26]:
def img_crawler(img_list):
    for idx,img_ in enumerate(img_list):
        r = requests.get(img_, stream=True, headers={'User-agent': 'Mozilla/5.0'})
        if r.status_code == 200:
            if idx % 10 == 0:
                print(f'{idx}....')
            with open(f'./img/{idx}.jpg', 'wb') as f:
                r.raw.decode_content = True
                shutil.copyfileobj(r.raw, f)
        else:
            print(r.status_code)

In [29]:
%%time
img_crawler(img_src[:100])

0....
10....
20....
30....
40....
50....
60....
70....
80....
90....
Wall time: 3.15 s


# 이미지를 다 load하지 못하는 것 같음

In [34]:
img_src[48]

'http://img2.tmon.kr/static2/img/blank.png'